In [1]:
import pandas as pd # データ分析
import numpy as np # 数値演算
import re # 正規表現
import sklearn # 機械学習
import xgboost as xgb # XGBoost
import seaborn as sns # 高性能グラフ
import matplotlib.pyplot as plt # グラフ
# Notebook内グラフ表示
%matplotlib inline

# インタラクティブなグラフ
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# 影響のない警告を非表示
import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
# SibSpとParchを組み合わせた新しい機能FamilySizeを作成する
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC # サポートベクターマシン
from sklearn.linear_model import LogisticRegression
##KA
# from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb

In [37]:
#pos = pd.read_csv('newtrain.csv', na_values = ' NaN', keep_default_na = True)
#neu = pd.read_csv('newtrain_s.csv', na_values = ' NaN', keep_default_na = True)
#neg = pd.read_csv('newtrain_y.csv', na_values = ' NaN', keep_default_na = True)

#pos["target"] = [0 for i in range(pos.shape[0])]
#neu["target"] = [1 for i in range(neu.shape[0])]
#neg["target"] = [2 for i in range(neg.shape[0])]
#train = pd.concat([pos, neu, neg], axis=0)#.dropna(thresh=15, axis=1)
#train.fillna(train.mean())


#train = train.sample(frac=1)

#train.to_csv("train_1.csv")
train = pd.read_csv('train_1.csv', na_values = ' NaN', keep_default_na = True)

train.head(3)

ID1   ID  BirthD  medicalD  age  Eye  Cornea Front K1  Cornea Front K2  \
0   660   51    1996      2013   17    0             42.3             44.4   
1  2225   38    1995      2013   18    1             42.2             43.0   
2   539  829    1991      2013   22    1             43.5             45.1   

   Cornea Front Km  Cornea Front Astig   ...    WFA Cornea 12  WFA Cornea 13  \
0             43.3                 2.1   ...            0.016          0.112   
1             42.6                 0.8   ...           -0.040          0.159   
2             44.3                 1.7   ...           -0.066          0.311   

   WFA Cornea 14  WFA Cornea 15  WFA Cornea 16  WFA Cornea 17  WFA Cornea 18  \
0          0.059          0.053         -0.006          0.021          0.012   
1          0.005         -0.056          0.037         -0.008         -0.010   
2          0.031          0.082         -0.033         -0.060          0.037   

   WFA Cornea 19  WFA Cornea 20  target  
0          0.006         -0.011       0  
1         -0.015          0.049       1  
2          0.050         -0.005       1  

[3 rows x 164 columns]

In [38]:

predictor = [x for x in train.columns if x not in ["target", "ID", "ID1", "BirthD", "CTSP 10mm", "PTI 0mm", "PTI 10mm", "WFA Corea 9", "WFA Corea 12", "WFA Corea 15", "WFA Corea 18"]]

print(train[predictor].info())

print(train.select_dtypes(include='object'))
print("------------")


features = train[predictor].values 
labels = train["target"].values





print(labels)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Columns: 157 entries, medicalD to WFA Cornea 20
dtypes: float64(135), int64(22)
memory usage: 2.9 MB
None
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[2381 rows x 0 columns]
------------
[0 1 1 ... 1 1 1]


In [39]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white')

In [40]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None, xgb = 0):
        params['random_state'] = seed
        if not xgb == 1:
          self.clf = clf(**params)
        else:
          self.clf = clf   
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

    def predict_proba(self, x):
        return self.clf.predict_proba(x)

    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
        return self.clf.fit(x,y).feature_importances_ ##KA

In [41]:
from imblearn.over_sampling import SMOTE
_N_CLASS = 3
def get_oof(clf, x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf):
    oof_train = np.zeros((ntrain, _N_CLASS))
    oof_test = np.zeros((ntest, _N_CLASS))
    oof_test_skf = np.empty((NFOLDS, ntest, _N_CLASS))

    # for i, (train_index, test_index) in enumerate(kf):
    for i, (train_index, test_index) in enumerate(kf.split(x_train)): ##KA
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        
        over_samples = SMOTE(random_state=1234) 
        x_tr, y_tr = over_samples.fit_sample(x_tr, y_tr)  
         
        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict_proba(x_te)
        oof_test_skf[i, :] = clf.predict_proba(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, _N_CLASS), oof_test.reshape(-1, _N_CLASS)


In [42]:
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
# Extra Treesのパラメータ
et_params = {
#    'n_jobs': -1,
#    'n_estimators':500,
    #'max_features': 0.5,
#    'max_depth': 8,
#    'min_samples_leaf': 2,
#    'verbose': 0
}

# AdaBoost parameters
# AdaBoostのパラメータ
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
# Gradient Boostingのパラメータ
gb_params = {
    'n_estimators': 180,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_split':180,
    'min_samples_leaf': 40,
    'verbose': 0,
    'learning_rate':0.1, 
    'random_state':10
}

# Support Vector Classifier parameters 
# Support Vector Classifierのパラメータ
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025,
    'probability':True
    }
lr_params = {
    
}
mlp_params = {
    'activation':'relu',
    'alpha':1e-05, 
    'batch_size':'auto',
    'beta_1':0.9, 
    'beta_2':0.999, 
    'early_stopping':False,
    'epsilon':1e-08, 
    'hidden_layer_sizes':(30, 3),
    'learning_rate':'constant', 
    'learning_rate_init':0.001,
    'max_iter':200, 
    'momentum':0.9, 
    'n_iter_no_change':10,
    'nesterovs_momentum':True, 
    'power_t':0.5, 
    'random_state':1,
    'shuffle':True, 
    'solver':'lbfgs', 
    'tol':0.0001,
    'verbose':False, 
    'warm_start':False
}

xgb_params = {
   'objective' : 'multi:softmax',
   'num_class' : 3,
   'max_depth': 6, 
   'lambda': 1.0, 
   'eta': 1.05, 
   'num_boost_round' : 100
}

lgbm_params = {'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 30,
          'max_depth': 3,
          'learning_rate': 0.1,
          'feature_fraction': 0.8,
          'min_child_samples': 60,
          'lambda_l2': 0.0675,
          'verbosity': -1,
         }

下面的代码是Stacking模型十折交叉验证之后的性能，每一个数就是一折的性能，最后是十折交叉验证的平均结果 

In [48]:


def accuracy(test_labels, pred_labels):
  correct1 = np.sum(test_labels == pred_labels) 
  n = len(test_labels)
  return float(correct1)/n


def test_stacking(x_train, y_train, x_test, y_test):
  ntrain = train.shape[0]
  ntest = test.shape[0]
  SEED = 0 # for reproducibility
  NFOLDS = 3 # set folds for out-of-fold prediction
  # kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)
  kf = KFold(n_splits=NFOLDS, shuffle=False, random_state=SEED) ##KA

#  rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
#  et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
#  ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
  gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
#  svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)
#  lr = SklearnHelper(clf=LogisticRegression, seed=SEED, params=lr_params)
#  mlp = SklearnHelper(clf=MLPClassifier, seed=SEED, params=mlp_params) 
  xgb_clf = SklearnHelper(clf=xgb.XGBClassifier(   objective = 'multi:softmax',
                                                   num_class = 3,
                                                   max_depth = 6, 
#                                                   lambda = 1.0, 
                                                   eta = 1.05, 
                                                   num_boost_round = 100), seed = SEED, xgb = 1, params=xgb_params)
#  lgbm = SklearnHelper(clf=lgb, seed=SEED, params=lgbm_params)
    
#  y_train = train['Survived'].ravel()
#  train = train.drop(['Survived'], axis=1)
  
#  x_train = train.values # Creates an array of the train data
#  x_test = test.values # Creats an array of the test data
  gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Gradient Boost
  xgb_oof_train, xgb_oof_test = get_oof(xgb_clf,x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Support Vector Classifier
#  et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Extra Trees
#  rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Random Forest
#  ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # AdaBoost 
#  svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Support Vector Classifier
#  lr_oof_train, lr_oof_test = get_oof(lr,x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Support Vector Classifier
#  mlp_oof_train, mlp_oof_test = get_oof(mlp,x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Support Vector Classifier
 # lgbm_oof_train, lgbm_oof_test = get_oof(lgb,x_train, y_train, x_test, ntrain, ntest, NFOLDS, kf) # Support Vector Classifier
  


  print("Training is complete") 


#  rf_feature = rf.feature_importances(x_train,y_train)
#  et_feature = et.feature_importances(x_train, y_train)
#  ada_feature = ada.feature_importances(x_train, y_train)
#  gb_feature = gb.feature_importances(x_train,y_train)


#  cols = train.columns.values
# Create a dataframe with features
# 特徴を使用してデータフレームを作成する
#  feature_dataframe = pd.DataFrame( {'features': predictor,
#     'Random Forest feature importances': rf_features,
#     'Extra Trees  feature importances': et_features,
#      'AdaBoost feature importances': ada_features,
#    'Gradient Boost feature importances': gb_features
#    })     

    
#  feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise

  #feature_dataframe.head(3)
#  feature_dataframe 

##########
#  base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
#     'ExtraTrees': et_oof_train.ravel(),
#     'AdaBoost': ada_oof_train.ravel(),
#      'GradientBoost': gb_oof_train.ravel()
#    })
#  base_predictions_train.head()


#  data = [
#    go.Heatmap(
#        z= base_predictions_train.astype(float).corr().values ,
#        x=base_predictions_train.columns.values,
#        y= base_predictions_train.columns.values,
#          colorscale='Viridis',
#            showscale=True,
#            reversescale = True
#    )
#]
#  py.iplot(data, filename='labelled-heatmap')
##########
  
  x_train = np.concatenate((gb_oof_train, xgb_oof_train), axis=1)
  x_test = np.concatenate((gb_oof_test, xgb_oof_test), axis=1)
    
#  gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
#   n_estimators= 2000,
#   max_depth= 4,
#   min_child_weight= 2,
   #gamma=1,
#   gamma=0.9,                        
#   subsample=0.8,
#   colsample_bytree=0.8,
#   objective= 'multi:softmax',
#   num_class = 3,
#   nthread= -1,
#   scale_pos_weight=1
#  ).fit(x_train, y_train)
  gbm = LogisticRegression().fit(x_train, y_train)


  predictions = gbm.predict(x_test)  
  print("score1:", np.array(accuracy(predictions, y_test)))  
  return np.array(accuracy(predictions, y_test))
#  score1 += np.array(accuracy(predictions, y_test))
#  print("score1:", score1)

from sklearn.model_selection import train_test_split, StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)
score1 = 0
for train, test in kfold.split(features, labels):
  score1 += test_stacking(features[train], labels[train], features[test], labels[test])
print("final result:", score1 / 10)    
    

Training is complete
score1: 0.9625
Training is complete
score1: 0.9458333333333333
Training is complete
score1: 0.9288702928870293
Training is complete
score1: 0.9369747899159664
Training is complete
score1: 0.957983193277311
Training is complete
score1: 0.9411764705882353
Training is complete
score1: 0.919831223628692
Training is complete
score1: 0.9578059071729957
Training is complete
score1: 0.9746835443037974
Training is complete
score1: 0.9367088607594937
final result: 0.9462367615866855


In [60]:
import copy
test = pd.read_csv('test.csv', na_values = ' NaN', keep_default_na = True)
n_predictor = [x for x in test.columns if x not in ["target", "ID", "ID1", "BirthD", "CTSP 10mm", "PTI 0mm", "PTI 10mm", "WFA Corea 9", "WFA Corea 12", "WFA Corea 15", "WFA Corea 18", "Remark"]]
test.fillna(test.mean())
test = test[n_predictor]
print(test.head())
print(test.info())
test = test.values


   medicalD  age  Eye  Cornea Front K1  Cornea Front K2  Cornea Front Km  \
0      2012   26    0             43.2             44.1             43.6   
1      2012   26    0             44.1             45.2             44.6   
2      2013   27    0             45.3             46.4             45.8   
3      2013   27    1             44.9             46.1             45.5   
4      2013   27    1             43.4             44.8             44.0   

   Cornea Front Astig  Cornea Front Q-val  Cornea Back K1  Cornea Back K2  \
0                 0.9               -0.37            -6.2            -6.5   
1                 1.1               -0.39            -6.3            -6.8   
2                 1.1               -0.33            -6.5            -6.8   
3                 1.2               -0.33            -6.4            -6.8   
4                 1.4               -0.42            -6.2            -6.6   

       ...        WFA Cornea 11  WFA Cornea 12  WFA Cornea 13  WFA Cornea 14  \


In [63]:
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
xgb_clf = SklearnHelper(clf=xgb.XGBClassifier(   objective = 'multi:softmax',
                                                   num_class = 3,
                                                   max_depth = 6, 
                                                   eta = 1.05, 
                                                   num_boost_round = 100), seed = SEED, xgb = 1, params=xgb_params)
over_samples = SMOTE(random_state=1234) 
x_tr, y_tr = over_samples.fit_sample(features, labels)  
gb.train(x_tr, y_tr)
xgb_clf.train(x_tr, y_tr)
gbm = LogisticRegression().fit(np.concatenate((gb.predict_proba(x_tr), xgb_clf.predict_proba(x_tr)), axis=1), y_tr)

gb_test = gb.predict_proba(test)
xgb_test = xgb_clf.predict_proba(test)
test_layer_1 = np.concatenate((gb_test, xgb_test), axis=1)
test_results = gbm.predict(test_layer_1)  


在测试集上进行预测

好吧 最后发现做不做Stacking在Test Set上的预测结果都是一样的。。。。

In [72]:
test_results

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 0, 0, 0, 0], dtype=int64)

In [73]:
len(test_results)

269

In [95]:
final_results = []
for item in test_results:
  if item == 1:
    final_results.append("疑似")
  elif item == 0:
    final_results.append("正常")
  elif item == 2:
    final_results.append("圆锥")
    

In [96]:
from pandas import Series
result = pd.read_csv('test.csv', na_values = ' NaN', keep_default_na = True, encoding='utf-8')
#result = pd.DataFrame(result)
result.insert(0, "Label", final_results, True) 
#result["Label"] = test_results
result.to_csv("test_result.csv", encoding="utf_8_sig")

In [46]:
SEED = 10

XGB单模型训练

In [47]:
param = {
#    'booster': 'gbtree',  
    'objective': 'multi:softmax',  # 多分类的问题 binary:logistic 
    'num_class': 3,        
#    'gamma': 0.005,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 6,               # 构建树的深度，越大越容易过拟合
    'lambda': 1.0,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
#    'subsample': 0.7,              # 随机采样训练样本
#    'colsample_bytree': 0.7,       # 生成树时进行的列采样
#    'min_child_weight': 3,
#    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 1.05,                  # 如同学习率
#    'seed': 1000,
#    'nthread': 4,                  # cpu 线程数
}


from sklearn.model_selection import train_test_split, StratifiedKFold

rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)
lr = SklearnHelper(clf=LogisticRegression, seed=SEED, params=lr_params)
mlp = SklearnHelper(clf=MLPClassifier, seed=SEED, params=mlp_params) 
xgb_clf = SklearnHelper(clf=xgb.XGBClassifier, seed=SEED, params=xgb_params)
    
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)
score1 = 0
for train, test in kfold.split(features, labels):
  f_train = xgb.DMatrix(features[train], label = labels[train])
  clf = xgb.train(param, f_train, num_boost_round = 200)
  score1 +=  np.array(accuracy(clf.predict(xgb.DMatrix(features[test])), labels[test]))

    #  xgb_clf.fit(features[train], labels[train])
#  score1 +=  np.array(accuracy(xgb_clf.predict(features[test]), labels[test])) 
print("final result:", score1 / 10)    

final result: 0.9437016294521623


XGB单模型的预测结果

In [65]:
single_xgb_clf = SklearnHelper(clf=xgb.XGBClassifier(   objective = 'multi:softmax',
                                                   num_class = 3,
                                                   max_depth = 6, 
                                                   eta = 1.05, 
                                                   num_boost_round = 100), seed = SEED, xgb = 1, params=xgb_params)
single_xgb_clf.train(features, labels)
single_xgb_clf.predict(test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 0, 0, 0, 0], dtype=int64)